# Utilities

This notebook contains utility code that is useful to multiple notebooks.

## Derivative Filter

In many control schemes that follow trajectories, higher order derivatives are often needed. This section discusses an implementation of a digital derivative based in Laplace system theory.

Recall from system theory that the Laplace transform is a useful way of analyzing linear time-invariant (LTI) systems. It takes time-domain systems and expresses them in the Laplace domain (a superset of the Fourier frequency domain). The unilateral Laplace transform is defined by
$$
\begin{equation}
F(s) = \int_0^\infty f(t) e^{-st} dt,
\end{equation}
$$
where $t$ is a time variable and $s$ is the Laplace variable. This transform can be thought as projecting a time-domain function onto exponentials of varying phase.

This transform is particularly useful when analyzing LTI systems of differential equations. In particular, the Laplace transform of the time-derivative of a continuous system $f(t)$ can be found to be
$$
\begin{equation}
\frac{d}{dt} f(t) \stackrel{\mathcal{L}}{\rightleftharpoons} sF(s).
\end{equation}
$$

Therefore, differentiation in the time-domain corresponds to multiplication by $s$ in the Laplace domain. Although a pure derivative in the Laplace domain is causal, it is not realizable (i.e., can you build an RLC circuit that realizes $F(s)=s$?). Therefore, a band-limited derivative (i.e., a *dirty derivative*) is used:
$$
\begin{equation}\label{eq:dirty-derivative}
G(s) = \frac{s}{\tau s + 1}.
\end{equation}
$$

From analog filter design, remember that a first-order low-pass filter with cutoff frequency $\omega_c$ is defined in the Laplace domain as
$$
\begin{equation}\label{eq:lpf}
H_{LPF}(s) = \frac{\omega_c}{s + \omega_c} = \frac{1}{s/\omega_c + 1},
\end{equation}
$$
and a first-order high-pass filter is
$$
\begin{equation}\label{eq:hpf}
H_{HPF}(s) = \frac{s}{s + \omega_c} = \frac{s/\omega_c}{s/\omega_c + 1}.
\end{equation}
$$

Note that the cutoff frequency is related to the filter time-constant (and RC circuits) by
$$
\begin{equation}
\tau = RC = \frac{1}{2\pi f_c} = \frac{1}{\omega_c}.
\end{equation}
$$

From this discussion, we can see that the dirty derivative \eqref{eq:dirty-derivative} can be thought of as a filtered version of a pure derivative with bandwidth $1/\tau$. This is useful as the LPF will prevent the derivative operator from picking up high-frequency components of the input signal and amplifying noise.

A common value of $\tau$ is $0.5$ ($20$ Hz). A higher $\tau$ corresponds to less bandwidth and more rejection of high-frequency components, which results in a smoother output.

### Digital Implementation

In order to implement a derivative filter in a computer, it of course must be discretized. The technique used here is to map $G(s)$ to the $z$-domain via the bilinear transform (AKA, Tustin approximation)
$$
\begin{equation}
s \mapsto \frac{2}{T}\frac{1-z^{-1}}{1+z^{-1}} ,
\end{equation}
$$
where $T$ is the sample period. Once we have an expression in the $z$-domain, the inverse $z$-transform can be used to give a discrete-time implementation of the dirty derivative.

#### Resources

- [SE.DSP: First-Derivative Analog Filter](https://dsp.stackexchange.com/questions/41109/first-derivative-analog-filter)
- [Blog: Causal, but not Realizable](http://blog.jafma.net/2015/10/04/differentiation-derivative-is-causal-but-not-exactly-realizable/)

In [1]:
class DirtyDerivative:
    """Dirty Derivative
    
    Provides a first-order derivative of a signal.
    
    This class creates a filtered derivative based on a
    band-limited low-pass filter with transfer function:
    
        G(s) = s/(tau*s + 1)
        
    This is done because a pure differentiator (D(s) = s)
    is not realizable.    
    """
    def __init__(self, tau=0.05):
        # time constant of dirty-derivative filter.
        # Higher leads to increased smoothing.
        self.tau = tau
        
        # internal memory for lagged signal value
        self.x_d1 = None
        
        # Current value of derivative
        self.dxdt = None
        
    def update(self, x, Ts):
        # Calculate digital derivative constants
        a1 = (2*self.tau - Ts)/(2*self.tau + Ts)
        a2 = 2/(2*self.tau + Ts)
        
        if self.x_d1 is None:
            self.x_d1 = np.zeros(x.shape)
            self.dxdt = np.zeros(x.shape)
        
        # calculate derivative
        self.dxdt = a1*self.dxdt + a2*(x - self.x_d1)
        
        # store value for next time
        self.x_d1 = x
        
        return self.dxdt